In [ ]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras import optimizers
from keras import initializers,regularizers

In [ ]:
#File importation
str0="ts_L60_Z12_A500_DX50_bias5_N10000"
fnamex ="DATA/x_" + str0 + ".csv"
fnamey ="DATA/y_" + str0 + ".csv"

x = np.loadtxt(fnamex,delimiter=',',dtype=float)
N = len(x)
print('Length of x = ',N)



categ = np.loadtxt(fnamey,delimiter=',', dtype=int)
#Number of categories
N_categ = 3
y = np.zeros((N,N_categ))

for n in range(N):
    y[n][categ[n]] = 1
    
print(y[0])



In [ ]:
#We saw that the nn was not converging and it might be
#due to some average, so we are going to remove it

xm = x.mean(axis=1)
for n in range(N):
    x[n] = x[n]-xm[n]
    
std = x.std(axis=1)
for n in range(N):
    x[n] = x[n] / std[n]  

In [ ]:
#Data visualization
plt.plot(x[0])
plt.plot(x[1])
plt.plot(x[2])
plt.show()

In [ ]:
#We asign the data for training and test
perc_train = 0.8
N_train = int(N * perc_train)
N_val = N-N_train

x_train = x[:N_train]
y_train = y[:N_train]

x_val = x[N_train:]
y_val = y[N_train:]

L =len(x[0])

print('Number of samples',N)
print('Training samples',N_train)
print('Validation samples',N_val)
print('Length of x[0]',L)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],L,1)
x_val = x_val.reshape(x_val.shape[0],L,1)
input_shape = (L,1)

In [ ]:
from keras import initializers,regularizers
np.random.seed(12345)

#We use l1 for Lazzo regularization
reg = regularizers.l1(0.01)    

#Now we initialize the NN weights
ini = initializers.RandomNormal(mean=0,stddev=0.05)

model = Sequential()
model.add(Conv1D(filters =5, kernel_size =11, 
                 kernel_regularizer = reg,
                 kernel_initializer = ini,
                 activation = 'relu',
                 input_shape =input_shape))

model.add(AveragePooling1D(5))
model.add(Conv1D(filters=5,kernel_size=7,activation="relu"))
model.add(Flatten())
model.add(Dense(10,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(N_categ,activation='softmax'))

print(model.summary())

from keras import optimizers

opt =optimizers.Adam()
#opt = optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov= True)

model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=opt,metrics=['accuracy'])

In [ ]:
batch_size =250
epochs = 100

fit = model.fit(x_train,y_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x_val,y_val),
               verbose=2, shuffle=True)

In [ ]:
for obs in ('accuracy','loss'):
    plt.figure(figsize=(6,4))
    plt.plot(fit.history[obs],'r',
            label =obs+' of training data')
    
    #dashed line!!
    plt.plot(fit.history['val_'+obs],'b--',
            label =obs+' of validation data')
    
    plt.ylabel(obs)
    plt.xlabel('epochs')
    plt.legend()
    plt.show()

## 2 
By reducing the signal-to-noise ratio, namely the
amplitude of the external signal in the data (A) with
respect to the step typical size DX of the jump process,
check where the network (defined in the class) starts to
fail discriminating the categories.


In [ ]:
def data_import(str0):
    fnamex ="DATA/x_" + str0 + ".csv"
    fnamey ="DATA/y_" + str0 + ".csv"

    x = np.loadtxt(fnamex,delimiter=',',dtype=float)
    N = len(x)
    print('Length of x = ',N)



    categ = np.loadtxt(fnamey,delimiter=',', dtype=int)
    #Number of categories
    N_categ = 3
    y = np.zeros((N,N_categ))

    for n in range(N):
        y[n][categ[n]] = 1
     
    
    return x,y,N
    

def set_spliting(N,x,y,perc_train = 0.8):  
    N_train = int(N * perc_train)
    N_val = N-N_train

    x_train = x[:N_train]
    y_train = y[:N_train]

    x_val = x[N_train:]
    y_val = y[N_train:]
    
    L =len(x[0])
    
    
    x_train = x_train.reshape(x_train.shape[0],L,1)
    x_val = x_val.reshape(x_val.shape[0],L,1)
    input_shape = (L,1)

    return x_train, y_train, x_val, y_val, input_shape
    
def data_process(x,N):
    xm = x.mean(axis=1)
    for n in range(N):
        x[n] = x[n]-xm[n]
    
    std = x.std(axis=1)
    for n in range(N):
        x[n] = x[n] / std[n]
    return x

def compilation(input_shape,N_categ=3):
    np.random.seed(12345)

    #We use l1 for Lazzo regularization
    reg = regularizers.l1(0.01)    

        #Now we initialize the NN weights
    ini = initializers.RandomNormal(mean=0,stddev=0.05)

    model = Sequential()
    model.add(Conv1D(filters =5, kernel_size =11, 
                     kernel_regularizer = reg,
                     kernel_initializer = ini,
                     activation = 'relu',
                     input_shape =input_shape))

    model.add(AveragePooling1D(5))
    model.add(Conv1D(filters=5,kernel_size=7,activation="relu"))
    model.add(Flatten())
    model.add(Dense(10,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(N_categ,activation='softmax'))

    print(model.summary())

   

    opt =optimizers.Adam()
    #opt = optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov= True)

    model.compile(loss=keras.losses.categorical_crossentropy,
                 optimizer=opt,metrics=['accuracy'])
    return model

In [ ]:
x1,y1,N1 = data_import("ts_L60_Z12_A10_DX50_bias5_N10000")
x2,y2,N2 = data_import("ts_L60_Z12_A500_DX50_bias5_N10000")
x3,y3,N3 = data_import("ts_L60_Z12_A1000_DX50_bias5_N10000")
x4,y4,N4 = data_import("ts_L60_Z12_A10000_DX50_bias5_N10000")


In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(nrows=4, ncols=1, figsize=(12, 12))

for j in range(20):
    ax1.plot([i for i in range(j*len(x1[0]),(len(x1[0])+j*len(x1[0])))],x1[j],color='red')
ax1.set_ylabel('Amplitude', fontsize=10)

for j in range(20):
    ax2.plot([i for i in range(j*len(x2[0]),(len(x2[0])+j*len(x2[0])))],x2[j],color='blue')
ax2.set_ylabel('Amplitude', fontsize=10)

for j in range(20):
    ax3.plot([i for i in range(j*len(x3[0]),(len(x3[0])+j*len(x3[0])))],x3[j],color='black')
ax3.set_ylabel('Amplitude', fontsize=10)

for j in range(20):
    ax4.plot([i for i in range(j*len(x4[0]),(len(x4[0])+j*len(x4[0])))],x4[j],color='orange')
ax4.set_xlabel('Time', fontsize=20)
ax4.set_ylabel('Amplitude', fontsize=10)
plt.show()

In [ ]:
x1 = data_process(x1,N1)
x2 = data_process(x2,N2)
x3 = data_process(x3,N3)
x4 = data_process(x4,N4)

x1_train, y1_train, x1_val, y1_val, input_shape1 = set_spliting(N1,x1,y1,perc_train = 0.8)
x2_train, y2_train, x2_val, y2_val, input_shape2 = set_spliting(N2,x2,y2,perc_train = 0.8)
x3_train, y3_train, x3_val, y3_val, input_shape3 = set_spliting(N3,x3,y3,perc_train = 0.8)
x4_train, y4_train, x4_val, y4_val, input_shape4 = set_spliting(N4,x4,y4,perc_train = 0.8)

In [ ]:
batch_size =250
epochs = 100

fit1 = compilation(input_shape1,N_categ=3).fit(x1_train,y1_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x1_val,y1_val),
               verbose=2, shuffle=True)

fit2 = compilation(input_shape2,N_categ=3).fit(x2_train,y2_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x2_val,y2_val),
               verbose=2, shuffle=True)

fit3 = compilation(input_shape3,N_categ=3).fit(x3_train,y3_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x3_val,y3_val),
               verbose=2, shuffle=True)

fit4 = compilation(input_shape4,N_categ=3).fit(x4_train,y4_train,
               batch_size=batch_size,
               epochs = epochs,
               validation_data =(x4_val,y4_val),
               verbose=2, shuffle=True)

In [ ]:
fig, ((ax1,ax2),(ax3,ax4),(ax5,ax6),(ax7,ax8)) = plt.subplots(nrows=4, ncols=2, figsize=(12, 20))

ax1.plot(fit1.history['accuracy'],'r', label ='Accuracy of training data')
ax1.plot(fit1.history['val_accuracy'],'b--', label ='Accuracy of validation data')
ax1.set_ylabel('Accuracy')
ax1.set_xlabel('epochs')
ax1.legend()

ax2.plot(fit1.history['loss'],'r', label ='Loss of training data')
ax2.plot(fit1.history['val_loss'],'b--', label ='Loss of validation data')
ax2.set_ylabel('Loss')
ax2.set_xlabel('epochs')
ax2.legend()



ax3.plot(fit2.history['accuracy'],'r', label ='Accuracy of training data')
ax3.plot(fit2.history['val_accuracy'],'b--', label ='Accuracy of validation data')
ax3.set_ylabel('Accuracy')
ax3.set_xlabel('epochs')
ax3.legend()

ax4.plot(fit2.history['loss'],'r', label ='Loss of training data')
ax4.plot(fit2.history['val_loss'],'b--', label ='Loss of validation data')
ax4.set_ylabel('Loss')
ax4.set_xlabel('epochs')
ax4.legend()



ax5.plot(fit3.history['accuracy'],'r', label ='Accuracy of training data')
ax5.plot(fit3.history['val_accuracy'],'b--', label ='Accuracy of validation data')
ax5.set_ylabel('Accuracy')
ax5.set_xlabel('epochs')
ax5.legend()

ax6.plot(fit3.history['loss'],'r', label ='Loss of training data')
ax6.plot(fit3.history['val_loss'],'b--', label ='Loss of validation data')
ax6.set_ylabel('Loss')
ax6.set_xlabel('epochs')
ax6.legend()



ax7.plot(fit4.history['accuracy'],'r', label ='Accuracy of training data')
ax7.plot(fit4.history['val_accuracy'],'b--', label ='Accuracy of validation data')
ax7.set_ylabel('Accuracy')
ax7.set_xlabel('epochs')
ax7.legend()

ax8.plot(fit4.history['loss'],'r', label ='Loss of training data')
ax8.plot(fit4.history['val_loss'],'b--', label ='Loss of validation data')
ax8.set_ylabel('Loss')
ax8.set_xlabel('epochs')
ax8.legend()

plt.show